### This notebook aims to:
1. Implement a simple logic to differentiate speakers in the interview transcript: _interviewer_ vs _interviewee_.
2. To develop a simple metric that can be used in (1).
---
In our approach we will utilise the list with "interviewer's" reference questions:
1. For each reference question, we separetly calculate the best matching scores for each spk_0's and spk_1's lines in the transcript.
2. Then for each speaker and each question we take the MAX mathcing score. Since we have a list of reference questions the output in this step is a vector with maximum matching scores. 
3. Then for each speaker we calculate metric by taking the average of maximum matching scores from step (2)
4. A speaker who has the highest metric (from (3)) is the interviewer, because his lines are best matched the reference list of questions.

In [1]:
import pandas as pd
import difflib  as dl
import numpy as np

In [2]:
# read a sample transcript which we saved in data frame
transcript = pd.read_excel("../res/processed_transcript_from_json_to_dataframe.xlsx")

In [3]:
transcript.head()

,Unnamed: 0,start_time,end_time,speaker,text,confidence
0,0,0:00:00.0,9.13,spk_0,And resources at the National University of Si...,"[{'text': 'And', 'confidence': 0.9411, 'start_..."
1,1,0:00:09.13,1.23,spk_1,[PII] nice to be here.,"[{'text': '[PII]', 'confidence': 0.9999, 'star..."
2,2,0:00:10.37,3.37,spk_0,Dr [PII] on general practitioner from Dr anywh...,"[{'text': 'Dr', 'confidence': 0.7912, 'start_t..."
3,3,0:00:13.75,2.07,spk_2,Thank you very much for inviting us here today.,"[{'text': 'Thank', 'confidence': 1.0, 'start_t..."
4,4,0:00:15.83,4.04,spk_0,And dr medical director at Felix Medical group.,"[{'text': 'And', 'confidence': 1.0, 'start_tim..."


### Assumptions:
1. spk_0 is always an interviewer
2. Other speakers are interviewees
3. In this project we combine spk_1 + spk_2 + spk_3 into one "person" as a "collective interviewee"

In [4]:
transcript['speaker'] = transcript['speaker'].astype(str)

In [5]:
transcript['speaker'] = transcript['speaker'].apply(lambda x:x if x=='spk_0' else 'spk_1')

In [6]:
transcript = transcript[['speaker','text']]
transcript.head(10)

,speaker,text
0,spk_0,And resources at the National University of Si...
1,spk_1,[PII] nice to be here.
2,spk_0,Dr [PII] on general practitioner from Dr anywh...
3,spk_1,Thank you very much for inviting us here today.
4,spk_0,And dr medical director at Felix Medical group.
5,spk_1,Good afternoon [PII] and everyone else.
6,spk_0,"Okay gentlemen, thanks so much for joining us...."
7,spk_1,I think healthier S. G. Is a whole rethinking ...
8,spk_0,And what do you think? Do you agree with that?
9,spk_1,Absolutely. I think [PII] has got the summary ...


#### This is a reference list of questions, for this interview
The reference list represents high level question for different topics and may varies from all the questioned the interviewer asked. Sometimes interviewer asks more questions for the same topic, or gives some background before asks a question. We will identify and map lines in transcrip to the reference questions and answers to these questions. 

In [7]:
list_of_questions =[
    "From your point of view What is Healthier SG",
    "when you look at our health care system, what was wrong with it? I mean, why are we doing this now? Why are we shifting in this direction?  ",
    "The idea of having family doctors is something so common elsewhere in the world. Why are we only doing it now?",
    "Will this also add on to increasing burdens for GPs? What would Healthier SG mean for GPs?"
    "How dispence of medications will affect your operations?",
    "Fees will be paid to GPS based on the health risk profiles of patients and depending on the outcomes.How are you going to measure this when outcome is going to be so different for everyone?",
    "As a national savings for the healthcare system, will we save money is this is running correctly?",
    "Should the typical patient be concerned about rising costs when they go see the doctor world prices increase over time?",
    "If there's one thing that you could you know change right now fixed right now with regards to our health care system, what would it be?"
]

### Step 1: to map reference questions  to our speakers in transcript


##### Process spk_0:

In [8]:
# Select only spk_0 lines:
df_spk0 = transcript[transcript.speaker=="spk_0"]

In [9]:
df_spk0=df_spk0.reset_index()

In [10]:
df_spk0.head()

,index,speaker,text
0,0,spk_0,And resources at the National University of Si...
1,2,spk_0,Dr [PII] on general practitioner from Dr anywh...
2,4,spk_0,And dr medical director at Felix Medical group.
3,6,spk_0,"Okay gentlemen, thanks so much for joining us...."
4,8,spk_0,And what do you think? Do you agree with that?


In [11]:
import difflib  as dl

In [12]:
possibilities = df_spk0.text.tolist()
# get max length for padding:
padding_len = max([len(val) for val in possibilities])
print(padding_len)

786


In [13]:
def get_similarity(s1,s2, padding_len):
    """
    s1: current line of speaker
    s2: is for question from the reference list of questions
    padding_len: all lines in transcript are aligned to the left side by inserting 
    padding to the right end of the string. This help to ensure a fair comparision.
    """
    # s1 need to be padded with 0:
    s1=s1.ljust(padding_len, '0')
    return dl.SequenceMatcher(None, s2, s1).ratio()
    
    

Getting a similarity scores for each question vs each line in transcript:

In [14]:
for i,q_i in enumerate(list_of_questions):
    df_spk0[q_i] = 0
    df_spk0[q_i] = df_spk0["text"].apply(lambda x:get_similarity(str(x), q_i,padding_len ))

In [15]:
df_spk0

,index,speaker,text,From your point of view What is Healthier SG,"when you look at our health care system, what was wrong with it? I mean, why are we doing this now? Why are we shifting in this direction?",The idea of having family doctors is something so common elsewhere in the world. Why are we only doing it now?,Will this also add on to increasing burdens for GPs? What would Healthier SG mean for GPs?How dispence of medications will affect your operations?,Fees will be paid to GPS based on the health risk profiles of patients and depending on the outcomes.How are you going to measure this when outcome is going to be so different for everyone?,"As a national savings for the healthcare system, will we save money is this is running correctly?",Should the typical patient be concerned about rising costs when they go see the doctor world prices increase over time?,"If there's one thing that you could you know change right now fixed right now with regards to our health care system, what would it be?"
0,0,spk_0,And resources at the National University of Si...,0.024096,0.041037,0.058036,0.075107,0.041026,0.056625,0.046409,0.026059
1,2,spk_0,Dr [PII] on general practitioner from Dr anywh...,0.026506,0.032397,0.031250,0.049356,0.053333,0.038505,0.050829,0.015201
2,4,spk_0,And dr medical director at Felix Medical group.,0.033735,0.034557,0.033482,0.040773,0.045128,0.029445,0.048619,0.021716
3,6,spk_0,"Okay gentlemen, thanks so much for joining us....",0.074699,0.019438,0.020089,0.025751,0.022564,0.015855,0.024309,0.004343
4,8,spk_0,And what do you think? Do you agree with that?,0.026506,0.036717,0.046875,0.027897,0.047179,0.036240,0.024309,0.030402
5,10,spk_0,"So from the sounds of it, it is quite a fundam...",0.031325,0.300216,0.069196,0.021459,0.028718,0.088335,0.035359,0.093377
6,12,spk_0,"it really is a preventive approach. You know, ...",0.048193,0.021598,0.013393,0.023605,0.043077,0.013590,0.030939,0.017372
7,14,spk_0,"why wait this long, is it only now? Because we...",0.024096,0.064795,0.245536,0.038627,0.028718,0.047565,0.050829,0.015201
8,16,spk_0,"[PII], what do you think to",0.024096,0.036717,0.033482,0.017167,0.036923,0.027180,0.019890,0.028230
9,18,spk_0,But but do we really have that ecosystem? I've...,0.021687,0.041037,0.055804,0.038627,0.041026,0.036240,0.041989,0.043431


For each Reference question (column in our case) calculate the max score:

In [20]:
spk_0_avg_score = np.mean(df_spk0.max().tolist()[3:])
spk_0_avg_score

0.20355017080803853

In [26]:
# add this score as a column to the transcript
transcript['spk_0_score']=spk_0_avg_score

#### Process spk_1:

In [21]:
# Select only interviewer's lines:
df_spk1 = transcript[transcript.speaker=="spk_1"]
df_spk1=df_spk1.reset_index()
df_spk1.head()

,index,speaker,text
0,1,spk_1,[PII] nice to be here.
1,3,spk_1,Thank you very much for inviting us here today.
2,5,spk_1,Good afternoon [PII] and everyone else.
3,7,spk_1,I think healthier S. G. Is a whole rethinking ...
4,9,spk_1,Absolutely. I think [PII] has got the summary ...


In [22]:
for i,q_i in enumerate(list_of_questions):
    df_spk1[q_i] = 0
    df_spk1[q_i] = df_spk1["text"].apply(lambda x:get_similarity(str(x), q_i,padding_len ))

In [23]:
df_spk1.head()

,index,speaker,text,From your point of view What is Healthier SG,"when you look at our health care system, what was wrong with it? I mean, why are we doing this now? Why are we shifting in this direction?",The idea of having family doctors is something so common elsewhere in the world. Why are we only doing it now?,Will this also add on to increasing burdens for GPs? What would Healthier SG mean for GPs?How dispence of medications will affect your operations?,Fees will be paid to GPS based on the health risk profiles of patients and depending on the outcomes.How are you going to measure this when outcome is going to be so different for everyone?,"As a national savings for the healthcare system, will we save money is this is running correctly?",Should the typical patient be concerned about rising costs when they go see the doctor world prices increase over time?,"If there's one thing that you could you know change right now fixed right now with regards to our health care system, what would it be?"
0,1,spk_1,[PII] nice to be here.,0.014458,0.019438,0.024554,0.015021,0.022564,0.024915,0.024309,0.015201
1,3,spk_1,Thank you very much for inviting us here today.,0.040964,0.045356,0.046875,0.038627,0.043077,0.038505,0.046409,0.043431
2,5,spk_1,Good afternoon [PII] and everyone else.,0.021687,0.019438,0.022321,0.040773,0.041026,0.036240,0.033149,0.023887
3,7,spk_1,I think healthier S. G. Is a whole rethinking ...,0.012048,0.025918,0.013393,0.023605,0.026667,0.022650,0.017680,0.021716
4,9,spk_1,Absolutely. I think [PII] has got the summary ...,0.016260,0.005900,0.003604,0.008230,0.005734,0.001211,0.001195,0.003550


In [24]:
spk_1_avg_score = np.mean(df_spk1.max().tolist()[3:])
spk_1_avg_score

0.05568236160918105

In [25]:
# add this score as a column to the transcript
transcript['spk_1_score']=spk_1_avg_score

### Compare the score for spk_0 and spk_1.
Those who has the highest score is "interviewer", because it's the transcript's lines best matched with the list of reference question

In [29]:
transcript.head(5)

,speaker,text,spk_1_score,spk_0_score
0,spk_0,And resources at the National University of Si...,0.055682,0.20355
1,spk_1,[PII] nice to be here.,0.055682,0.20355
2,spk_0,Dr [PII] on general practitioner from Dr anywh...,0.055682,0.20355
3,spk_1,Thank you very much for inviting us here today.,0.055682,0.20355
4,spk_0,And dr medical director at Felix Medical group.,0.055682,0.20355


In [30]:
def who_is_interviewer(df,i):
    # if the i-line is for spk_0 and spk_0_score>spk_1_score then spk_0 is interviewer
    if df['speaker'][i]=="spk_0":
        if df["spk_0_score"][i]>=df["spk_1_score"][i]:
            return "interviewer"
        else:
            return "interviewee"
    else: # the line i belongs to spk_1
        if df["spk_0_score"][i]>=df["spk_1_score"][i]:
            return "interviewee"
        else:
            return "interviewer"

In [33]:
# create a new column to keep values who is interviewee/interviewer:
transcript['role']=""
for ind in transcript.index:
    transcript['role'][ind] = who_is_interviewer(transcript, ind)

/Users/yana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [34]:
transcript

,speaker,text,spk_1_score,spk_0_score,role
0,spk_0,And resources at the National University of Si...,0.055682,0.20355,interviewer
1,spk_1,[PII] nice to be here.,0.055682,0.20355,interviewee
2,spk_0,Dr [PII] on general practitioner from Dr anywh...,0.055682,0.20355,interviewer
3,spk_1,Thank you very much for inviting us here today.,0.055682,0.20355,interviewee
4,spk_0,And dr medical director at Felix Medical group.,0.055682,0.20355,interviewer
...,...,...,...,...,...
61,spk_1,I will say vaccinations. You know the accessib...,0.055682,0.20355,interviewee
62,spk_0,also the flick of switch and everyone is tomorrow,0.055682,0.20355,interviewer
63,spk_1,somehow.,0.055682,0.20355,interviewee
64,spk_1,"Well, I wish we had done all of this five or s...",0.055682,0.20355,interviewee


### Saving processed results
This results are ready for text processing with AWS 

In [35]:
transcript.to_csv("../res/transcript_with_differentiated_interviewer_interviewee.csv",index=False)

moving to AWS SageMaker